#**Custom hàm xếp hạng các câu truy vấn**

##**Các hàm tách chữ trong query**

Hàm tìm kiếm theo từng chữ

In [ ]:
def one_token_query(text):
  list_txt = text.split()
  result = []
  for txt in list_txt:
    result.append(" "+ txt+" ")
  return result

Hàm tìm kiếm hai chữ liên tiếp nhau

In [ ]:
def two_token_query(text):
  txt_list = text.split()
  result = []
  while len(txt_list)>0:
    result.append(" " + " ".join(txt_list[:2]) + " ")
    txt_list = txt_list[1:]
    if len(txt_list) < 2:
      break
  return result

Hàm tìm kiếm ba chữ liên tiếp nhau

In [ ]:
def three_token_query(text):
  txt_list = text.split()
  result = []
  while len(txt_list)>0:
    result.append(" " + " ".join(txt_list[:3]) + " ")
    txt_list = txt_list[1:]
    if len(txt_list) < 3:
      break
  return result

Hàm tìm kiếm bốn chữ liên tiếp nhau

In [ ]:
def four_token_query(text):
  txt_list = text.split()
  result = []
  while len(txt_list)>0:
    result.append(" " + " ".join(txt_list[:4]) + " ")
    txt_list = txt_list[1:]
    if len(txt_list) < 4:
      break
  return result

Hàm tìm kiếm năm chữ liên tiếp nhau

In [ ]:
def five_token_query(text):
  txt_list = text.split()
  result = []
  while len(txt_list)>0:
    result.append(" " + " ".join(txt_list[:5]) + " ")
    txt_list = txt_list[1:]
    if len(txt_list) < 5:
      break
  return result

Hàm tìm kiếm sáu chữ liên tiếp nhau

In [ ]:
def six_token_query(text):
  txt_list = text.split()
  result = []
  while len(txt_list)>0:
    result.append(" " + " ".join(txt_list[:6]) + " ")
    txt_list = txt_list[1:]
    if len(txt_list) < 6:
      break
  return result

##**Hàm đánh giá điểm**

Ý tưởng chung của hàm là nếu số lượng chữ liên tiếp nhau trong query có xuất hiện trong lyric bất kì càng nhiều thì điểm của lyric tương ứng với query đó càng cao.

In [ ]:
def get_score(lyrics, query):
  scores =[]
  for lr in lyrics:
    lyric = " " + lr + " "
    score = 0
    one = one_token_query(query)
    for o in one:
      if o in lyric:
        score = score + 1
    two = two_token_query(query)

    for t in two:
      if t in lyric:
        score = score + 4
    three = three_token_query(query)

    for t in three:
      if t in lyric:
        score = score + 16
    four = four_token_query(query)
  
    for f in four:
      if f in lyric:
        score = score + 64
    five = five_token_query(query)
   
    for f in five:
      if f in lyric:
        score = score + 256
    six = six_token_query(query)
   
    for s in six:
      if s in lyric:
        score = score + 1024

    if query in lyric:
      score = score + 100000
    scores.append(score)
  return scores

##**Hàm lấy ra top 10 lyrics có sự tương đồng lớn nhất**

In [ ]:
def get_ranking(scores, song_id):
  list_rs = sorted(range(len(scores)), key=scores.__getitem__)
  list_rs.reverse()
  results = []
  for i in range(10):
    rs = "0000" + str(song_id[list_rs[i]])
    results.append(rs[-4:])
  return results

#**Download data**

In [ ]:
import json

#Download lyrics
!gdown --id 1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
#Download queries
!gdown --id 1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx

Downloading...
From: https://drive.google.com/uc?id=1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
To: /content/2335_Songs.json
100% 4.33M/4.33M [00:00<00:00, 189MB/s]
Downloading...
From: https://drive.google.com/uc?id=1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx
To: /content/500_queries.json
100% 39.0k/39.0k [00:00<00:00, 44.8MB/s]


#**Preliminaries & Corpus Creation**

In [ ]:
import os
import pprint as pp
import numpy as np
import json
import pandas as pd
df = pd.read_json (r'/content/2335_Songs.json')
df.to_csv (r'/content/sample_data/data.csv', index = None)
song_id = list(df["song_id"])

In [ ]:
from tqdm import tqdm
# link to data
CORPUS_SONG_JSON = "2335_Songs.json"
QUERIES_JSON = "500_queries.json"

In [ ]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as fo:
  corpus_data = json.load(fo)
with open(QUERIES_JSON, 'r', encoding='utf-8') as fo:
  queries_test = json.load(fo)

#**Pre-processing**

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.1 MB/s 


1. Chuyển tiếng Việt về dạng không dấu
2. Chuyển chữ HOA thành chữ thường


In [ ]:
import unidecode
for i,corpus in enumerate(corpus_data):
  corpus_data[i]['lyrics'] = unidecode.unidecode((corpus['title'] + " " + corpus['lyrics']).lower())

In [ ]:
for i,corpus in enumerate(corpus_data):
  corpus_data[i]['lyrics'] = (corpus['title'] +" " + corpus['lyrics']).lower()

Loại bỏ những ký tự không cần thiết

In [ ]:
strash_sign = '!"#$%&()*+,-./:’”;<=>?@[\\]^_`{|}~\t\n'
for i,corpus in enumerate(corpus_data):
  for sign in strash_sign:
    corpus_data[i]['lyrics'] = corpus['lyrics'].replace(sign, " ")

Loại bỏ những từ không cần thiết

In [ ]:
stop_word = ['intro', 'verse', 'hook', 'chorus', 'pre-chorus','bai hat']  #, ' anh ', ' em ', ' la ', ' thi ', ' nhung ', ' voi ', ' cua ' , ' a ', ' e ']
for i,corpus in enumerate(corpus_data):
  for word in stop_word:
    corpus_data[i]['lyrics'] = corpus['lyrics'].replace(word, " ")


Chuyển số về định dạng chữ

In [ ]:
for i,corpus in enumerate(corpus_data):
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 0 ", " khong ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 1 ", " mot ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 2 ", " hai ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 3 ", " ba ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 4 ", " bon ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 5 ", " nam ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 6 ", " sau ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 7 ", " bay ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 8 ", " tam ")
  corpus_data[i]['lyrics'] = corpus['lyrics'].replace(" 9 ", " chin ")
  corpus_data[i]['lyrics'] = " ".join(corpus['lyrics'].split())

In [ ]:
#Danh sách bài hát thiếu từ
lst = []
id = []
for i in range(2335):
  if '*' in corpus_data[i]['lyrics']:
    id.append(i)
    lst.append(corpus_data[i]['song_id'])


Thêm tất cả các lyrics vào một List

In [ ]:
corpus_list = []
for c in corpus_data:
  corpus_list.append(c["lyrics"])

In [ ]:
corpus_list[0:10]

['tình yêu thuỷ chung tinh yeu thuy chung tinh yeu thuy chung tran binh viet linh co phai la duyen khi anh gap em tu kiep truoc the uoc chung doi cung han hoan nhin nhau dam duoi long quyen luyen chuyen mai khong thoi em tu noi dau da den noi nay anh han hoan trao em vong tay va doi ta song chung hanh phuc cung xay dap cho moi duyen nay em oi em oi du phong ba du bao to du nang chay anh van yeu em mai mai khong thoi anh oi anh oi du gian nan du khon kho ta van yeu nhau den cuoi cuoc doi ta van ben nhau qua bao buon vui ta yeu nhau tinh yeu dam say tinh yeu ta dep tinh yeu tham thiet hanh phuc tram nam tinh yeu thuy chung',
 'lá thư của cha la thu cua cha la thu cua cha tran binh tu ngay con ra di cha long buon thuong nho co don va sau muon bao ngay thang mong cho tu ngay con ra di toi thoi gian bay gio tham thoat muoi hai nam chan cha thuong dung ngoi ngan ngo dk me con khong con nua di roi quen tu gia cha mot minh thui thui lanh gia va don coi nay con o phuong xa dung quen noi que nha

In [ ]:
import csv
import re

with open("submission.csv", "w") as csv_writer:
  fieldnames = ['query_id', 'song_id']
  writer = csv.DictWriter(csv_writer, fieldnames=fieldnames)
  writer.writeheader()
  i = 0
  for quess in tqdm(queries_test):
    ques = quess["query"]
    #Chuyển các queries về dạng chữ thường, không dấu
    preprocess_query = " " + unidecode.unidecode(ques.lower()) + " "
    #Loại bỏ các ký tự và từ ngữ không cần thiết
    for sign in strash_sign:
      preprocess_query = preprocess_query.replace(sign, " ")
    for word in stop_word:
      preprocess_query = preprocess_query.replace(word, " ")
    #Sửa các lỗi chính tả, teencode hay sai trong tiếng Việt
    preprocess_query = preprocess_query.replace(" nk ", " nh ")
    preprocess_query = preprocess_query.replace(" ik ", " di ")
    preprocess_query = preprocess_query.replace(" khox ", " khoc ")
    preprocess_query = preprocess_query.replace(" nayd ", " nay ")
    preprocess_query = preprocess_query.replace(" ko ", " khong ")
    preprocess_query = preprocess_query.replace(" k0 ", " khong ")
    preprocess_query = preprocess_query.replace(" k ", " khong ")
    preprocess_query = preprocess_query.replace(" khum ", " khong ")
    preprocess_query = preprocess_query.replace(" m ", " may ")
    preprocess_query = preprocess_query.replace(" t ", " tao ")
    preprocess_query = preprocess_query.replace(" bik ", " biet ")
    preprocess_query = preprocess_query.replace(" 0 ", " khong ")
    preprocess_query = preprocess_query.replace(" 1 ", " mot ")
    preprocess_query = preprocess_query.replace(" 2 ", " hai ")
    preprocess_query = preprocess_query.replace(" 3 ", " ba ")
    preprocess_query = preprocess_query.replace(" 4 ", " bon ")
    preprocess_query = preprocess_query.replace(" 5 ", " nam ")
    preprocess_query = preprocess_query.replace(" 6 ", " sau ")
    preprocess_query = preprocess_query.replace(" 7 ", " bay ")
    preprocess_query = preprocess_query.replace(" 8 ", " tam ")
    preprocess_query = preprocess_query.replace(" 9 ", " chin ")
    preprocess_query = preprocess_query.replace(" dzia ", " ve ")
    preprocess_query = preprocess_query.replace(" dze ", " ve ")
    preprocess_query = preprocess_query.replace(" dzo ", " vo ")
    preprocess_query = preprocess_query.replace(" wit ", " with ")
    preprocess_query = preprocess_query.replace(" ma gang ", " my gang ")
    preprocess_query = preprocess_query.replace(" mik ", " minh ")
    preprocess_query = " ".join(preprocess_query.split())

    #Tính điểm cho từng query
    scores = get_score(corpus_list, preprocess_query)
    results = get_ranking(scores, song_id)

    writer.writerow({'query_id': queries_test[i]["query_id"], "song_id": " ".join(results)})
    i = i+1
    

100%|██████████| 500/500 [02:17<00:00,  3.64it/s]
